In [133]:
import pandas as pd
import numpy as np
import os
from math import pi, atan, sinh, cos, log, tan, radians, degrees
import numpy as np
from PIL import Image,ImageDraw
from datetime import datetime,timedelta


In [49]:
zoom = 8.47
image_size = (719,719)

In [50]:

## extract the frames data
def latlon_to_pixel(lat, lon, zoom):
    """Convert latitude and longitude to pixel coordinates at the specified zoom level."""
    lat_rad = radians(lat)
    n = 2.0 ** zoom
    x = (lon + 180.0) / 360.0 * n * 256.0
    y = (1.0 - log(tan(lat_rad) + (1 / cos(lat_rad))) / pi) / 2.0 * n * 256.0
    return x, y

def pixel_to_latlon(x, y, zoom):
    """Convert pixel coordinates to latitude and longitude at the specified zoom level."""
    n = 2.0 ** zoom
    lon = x / (n * 256.0) * 360.0 - 180.0
    lat_rad = atan(sinh(pi * (1 - 2 * y / (n * 256.0))))
    lat = degrees(lat_rad)
    return lat, lon

def generate_latlon_grid(center_lat, center_lon, zoom, image_size):
    """Generate a grid of latitude and longitude values for each pixel in the image."""
    width, height = image_size

    # Get pixel coordinates of the center
    center_x, center_y = latlon_to_pixel(center_lat, center_lon, zoom)

    # Calculate pixel offsets
    half_width = width // 2
    half_height = height // 2

    latlon_grid = np.zeros((height, width, 2))

    for y in range(height):
        for x in range(width):
            pixel_x = center_x - half_width + x
            pixel_y = center_y - half_height + y
            lat, lon = pixel_to_latlon(pixel_x, pixel_y, zoom)
            latlon_grid[y, x] = (lat, lon)
    
    return latlon_grid

def rgb_to_hex(rgb):
    """Convert an RGB tuple to a hex string."""
    return '#{:02x}{:02x}{:02x}'.format(rgb[0], rgb[1], rgb[2])



In [51]:
image = Image.open('/home/tensor/vaib/imd_radar/target_processed/delhi/ppi/ppi_delhi_20240813T1340.png')
image = image.resize(image_size)  # Ensure the image is 512x512 pixels
pixels = np.array(image)
latlon_grid = generate_latlon_grid(28.7041, 77.1025, zoom, image_size)
data = []
# Print latitude, longitude, and hex color value of every pixel
for y in range(image_size[1]):
    for x in range(image_size[0]):
        lat, lon = latlon_grid[y, x]
        color = pixels[y, x]
        # hex_color = rgb_to_hex(color)
        data.append([x, y, lat, lon, color])

df = pd.DataFrame(data, columns=['Pixel_X', 'Pixel_Y', 'latitude', 'longitude', 'Color'])

In [52]:
df['lat_diff'] = (df['latitude'] - 28.5433498).abs()
df['lon_diff'] = (df['longitude'] - 77.187352).abs()
df['omega'] = df['lat_diff'] + df['lon_diff']
# df['localityId'] = row['localityId']
df.sort_values('omega')[:1]

,Pixel_X,Pixel_Y,latitude,longitude,Color,lat_diff,lon_diff,omega
291575,380,405,28.543966,77.185783,"[0, 0, 0, 0]",0.000616,0.001569,0.002185


In [129]:
img = Image.open('/home/tensor/vaib/imd_radar/target_processed/delhi/caz/caz_delhi_20240813T1530.png')


img = img.convert("RGBA")
draw = ImageDraw.Draw(img)
x, y = 275, 267 # Replace with your coordinates
radius = 0.5
pixels = img.load()
draw.ellipse((x-radius, y-radius, x+radius, y+radius), fill=(0, 0, 0))

img.save('/home/tensor/vaib/imd_radar/target_processed/delhi/caz/caz_delhi_20240813T1530eeeee.png')

IIT Delhi | 0.003128
CP Central Garden 28.6440, 77.1885 | 0.004157
Noida GIP mall 28.5675, 77.3262 | 0.003126
Pacific Mall Punjabi Baug 28.6417, 77.1067 | 0.001547
 

In [36]:
np.average([0.003128,0.004157,0.003126,0.001547])

np.float64(0.0029895)

In [128]:
import math

def lat_lon_to_pixel(latitude, longitude, zoom, img_width, img_height, center_lat, center_lon):
    # Constants
    origin_shift = 2 * math.pi * 6378137 / 2.0
    initial_resolution = 2 * math.pi * 6378137 / 256.0
    
    # Convert latitude/longitude to Mercator coordinates
    def lat_lon_to_meters(lat, lon):
        mx = lon * origin_shift / 180.0
        my = math.log(math.tan((90 + lat) * math.pi / 360.0)) / (math.pi / 180.0)
        my = my * origin_shift / 180.0
        return mx, my

    # Get the map center in Mercator coordinates
    center_mx, center_my = lat_lon_to_meters(center_lat, center_lon)
    
    # Convert the given latitude/longitude to Mercator coordinates
    mx, my = lat_lon_to_meters(latitude, longitude)
    
    # Calculate the resolution (meters per pixel) for the given zoom level
    resolution = initial_resolution / (2 ** zoom)
    
    # Calculate pixel offsets from the center
    pixel_x_offset = (mx - center_mx) / resolution
    pixel_y_offset = (center_my - my) / resolution
    
    # Calculate pixel coordinates relative to the top-left corner of the image
    pixel_x = img_width / 2 + pixel_x_offset
    pixel_y = img_height / 2 + pixel_y_offset
    
    # Round to nearest pixel
    pixel_x = int(round(pixel_x))
    pixel_y = int(round(pixel_y))
    
    return pixel_x, pixel_y

# Example usage:
latitude = 28.6440  # Replace with the latitude
longitude =  77.1885 # Replace with the longitude
zoom_level = 8.47
img_width = 566
img_height = 566
center_lat = 28.58967  # Center latitude
center_lon = 77.22196  # Center longitude

pixel_x, pixel_y = lat_lon_to_pixel(latitude, longitude, zoom_level, img_width, img_height, center_lat, center_lon)
print(f"Pixel coordinates: ({pixel_x}, {pixel_y})")

Pixel coordinates: (275, 267)


In [ ]:
# 28.5433498, 77.187352

import math

def lat_lon_to_pixel(latitude, longitude, zoom, img_width, img_height, center_lat, center_lon):
    # Constants
    origin_shift = 2 * math.pi * 6378137 / 2.0
    initial_resolution = 2 * math.pi * 6378137 / 256.0
    
    # Convert latitude/longitude to Mercator coordinates
    def lat_lon_to_meters(lat, lon):
        mx = lon * origin_shift / 180.0
        my = math.log(math.tan((90 + lat) * math.pi / 360.0)) / (math.pi / 180.0)
        my = my * origin_shift / 180.0
        return mx, my

    # Get the map center in Mercator coordinates
    center_mx, center_my = lat_lon_to_meters(center_lat, center_lon)
    
    # Convert the given latitude/longitude to Mercator coordinates
    mx, my = lat_lon_to_meters(latitude, longitude)
    
    # Calculate the resolution (meters per pixel) for the given zoom level
    resolution = initial_resolution / (2 ** zoom)
    
    # Calculate pixel offsets from the center
    pixel_x_offset = (mx - center_mx) / resolution
    pixel_y_offset = (center_my - my) / resolution
    
    # Calculate pixel coordinates relative to the top-left corner of the image
    pixel_x = img_width / 2 + pixel_x_offset
    pixel_y = img_height / 2 + pixel_y_offset
    
    # Round to nearest pixel
    pixel_x = int(round(pixel_x))
    pixel_y = int(round(pixel_y))
    
    return pixel_x, pixel_y

In [79]:
def get_color_from_image(file_name,x,y,image_size):
    image = Image.open(file_name)
    image = image.resize(image_size)  
    pixels = np.array(image)
    color = pixels[x,y]
    return color

In [130]:
image_widths = {'caz':566,'ppi':719,'sri':719}
zoom_levels = {'caz':8.47,'ppi':9.54,'sri':9.54}
center_lat = 28.58967  # Center latitude
center_lon = 77.22196  # Center longitude

In [135]:
## get list of all the processed images
all_processed = []
home_path = '/home/tensor/vaib/imd_radar'
processed_source = os.path.join(home_path,'target_processed')

for location in os.listdir(processed_source):
    if location == 'delhi':
        for variable in os.listdir(os.path.join(processed_source,location)):
            for file in os.listdir(os.path.join(processed_source,location,variable)):
                if file not in all_processed:
                    all_processed.append(file)
df_imgs = pd.DataFrame({'files':all_processed})

In [131]:
def get_timestamp(x):
    return datetime.strptime(x,'%Y%m%dT%H%M')

In [136]:
# df_processed
df_imgs['var'] = df_imgs['files'].str.split("_").str[0]
df_imgs['location'] = df_imgs['files'].str.split("_").str[1]
df_imgs['timestamp'] = df_imgs['files'].str.split("_").str[2].str[:-4]
df_imgs['timestamp'] = df_imgs['timestamp'].apply(get_timestamp)


In [138]:
df_imgs.sort_values('timestamp',ascending=False,inplace=True)

In [143]:
df_imgs['file_path'] = processed_source + "/" + df_imgs['location'] +"/" + df_imgs['var']+"/" + df_imgs['files']


'/home/tensor/vaib/imd_radar/target_processed/delhi/caz/caz_delhi_20240812T2110.png'